# Projektarbeit Weather Forecast
1. Daten Filtern und Aufbereiten
2. Besten Parameter für den Random Forest
3. Ergebnis Analysieren

In [1]:
# Import der Bibliotheken
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as sl

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn import neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn import linear_model

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso


%matplotlib inline
import scipy as sc
import math as ma
from scipy import linalg, optimize, constants, interpolate, special, stats
from math import exp, pow, sqrt, log

import seaborn as sns #spezielle Graphikdarstellungen
import statsmodels.api as sm
import statsmodels.stats.api as sms

## Daten Laden
-
Zunächst laden wir die csv dateien in ein Pandas Dataframe auf dem die Daten im preprocessing verarbeitet werden können.
Anschließend definieren wir eine Funktion `filter_columns`.
Diese Funktion ermöglicht die Transformation von Daten in einem Pandas DataFrame der Daten, 
wobei zeitabhängige Verzögerungen und Varianten für bestimmte Spalten berücksichtigt werden. Die Funktion erfordert folgende Parameter:

- `columns`: Eine Liste von Spalten, die für die zeitabhängigen Varianten verwendet werden sollen, einschließlich der Zielspalte.
- `variants`: Die Anzahl der gewünschten zeitlichen Varianten.
- `extras`: Eine Liste der Zielspalten, für die eine spezielle zeitliche Variante die abhähngig der `variants` und `timelag` in der Zukunft liegt.
- `selected_pass`: Eine optionale Liste von Spalten, die unverändert bleiben sollen und ohne Kopien oder zeitliche Verschiebungen durch die Funktion durchgeschleust werden.
- `timelag`: Ein optionaler Parameter, der die zeitliche Verzögerung für ausgewählte Spalten festlegt in `extras` zu den ersten Daten gibt.

Die Funktion erstellt für die angegebenen Spalten zeitabhängige Varianten entsprechend der definierten Varianten und Verschiebungen. Zusätzlich werden spezielle Varianten für die in extras aufgeführten Zielspalten erstellt, wobei eine zusätzliche zeitliche Verschiebung durch timelag erfolgt. Der resultierende DataFrame wird zurückgegeben, wobei die in selected_pass aufgeführten Spalten unverändert bleiben.

In [2]:
straubingDaten = pd.read_csv("../data/Straubing.csv")
arberDaten = pd.read_csv("../data/Arber.csv")
schorndorfDaten = pd.read_csv("../data/Schorndorf.csv")

# Spaltennamen trimmen
straubingDaten.columns = straubingDaten.columns.str.strip()
arberDaten.columns = arberDaten.columns.str.strip()
schorndorfDaten.columns = schorndorfDaten.columns.str.strip()

print(straubingDaten.columns)
print(arberDaten.columns)
print(schorndorfDaten.columns)

Index(['DATE', 'MESS_DATUM', 'QUALITAETS_NIVEAU', 'LUFTTEMPERATUR',
       'DAMPFDRUCK', 'BEDECKUNGSGRAD', 'LUFTDRUCK_STATIONSHOEHE',
       'REL_FEUCHTE', 'WINDGESCHWINDIGKEIT', 'LUFTTEMPERATUR_MAXIMUM',
       'LUFTTEMPERATUR_MINIMUM', 'LUFTTEMP_AM_ERDB_MINIMUM',
       'WINDSPITZE_MAXIMUM', 'NIEDERSCHLAGSHOEHE', 'NIEDERSCHLAGSHOEHE_IND',
       'SONNENSCHEINDAUER', 'SCHNEEHOEHE'],
      dtype='object')
Index(['DATE', 'MESS_DATUM', 'QUALITAETS_NIVEAU', 'LUFTTEMPERATUR',
       'DAMPFDRUCK', 'BEDECKUNGSGRAD', 'LUFTDRUCK_STATIONSHOEHE',
       'REL_FEUCHTE', 'WINDGESCHWINDIGKEIT', 'LUFTTEMPERATUR_MAXIMUM',
       'LUFTTEMPERATUR_MINIMUM', 'LUFTTEMP_AM_ERDB_MINIMUM',
       'WINDSPITZE_MAXIMUM', 'NIEDERSCHLAGSHOEHE', 'NIEDERSCHLAGSHOEHE_IND',
       'SONNENSCHEINDAUER', 'SCHNEEHOEHE'],
      dtype='object')
Index(['DATE', 'MESS_DATUM', 'QUALITAETS_NIVEAU', 'LUFTTEMPERATUR',
       'DAMPFDRUCK', 'BEDECKUNGSGRAD', 'LUFTDRUCK_STATIONSHOEHE',
       'REL_FEUCHTE', 'WINDGESCHWINDIGKEIT', 'LU

In [3]:
def filter_columns(df, columns, variants=3, extras=None, selected_pass=None, timelag=0):
    if extras is None:
        extras = []
    if selected_pass is None:
        selected_pass = []
    
    # suffix für die spalten hinzufügen
    df = df.rename(columns=lambda x: f'{x}_0' if x not in selected_pass else x)

    # spalten kopieren und schiften um 1
    for v in range(1, variants + 1):
        for i in range(len(columns)):
            df[f'{columns[i]}_{v}'] = df[f'{columns[i]}_{v-1}'].shift(-1, fill_value=-999)
            if columns[i] in extras and v == variants:
                df[f'{columns[i]}_{v+1}'] = df[f'{columns[i]}_{v}'].shift(-1-timelag, fill_value=-999)

    # Alles wieder zusammensetzen
    columns_with_variants = [f'{col}_{v}' for col in columns for v in range(0, variants + 1)] + [f'{col}_{variants+1}' for col in extras]
    df_filtered = df[selected_pass + columns_with_variants]

    # 
    return df_filtered

In [8]:
def replace_missing_values(value, df, row, col):
    if value != -999:
        return value

    def replacement_logic(column, row, col):
        nonlocal value

        distances = np.abs(np.arange(len(column)) - row)

        non_missing_values = column[column != -999]

        if non_missing_values.size > 0:
            weights = np.exp(-0.1 * distances[:len(non_missing_values)])
            weighted_mean = np.sum(non_missing_values * weights) / np.sum(weights)
            return weighted_mean

        return -999

    return replacement_logic(df.iloc[:, col].values, row, col)

### Configuration zur Filterung
Zunächst wird die Filterung Configuriert:
- `selected_columns`: Eine Liste von Spalten, die für die zeitabhängigen Varianten verwendet werden sollen, einschließlich der Zielspalte.
- `selected_result`: Eine Liste der Zielspalten.
- `selected_pass`: Eine Liste an Spalten die keinen lediglich zum joinen von den drei datensätzen ist, diese Spalten werden später wieder rausgenommen.
- `vergangenheit`: Der Vergangenheitswert gibt an wie weit in die vergangenheit die Daten mitgenommen wird.
- `timelag`: Der Timelag gibt an wie weit in der zukunft die Prognose liegen muss, abhängig vom neuesten Datensatz.

In [9]:
selected_columns = ['LUFTTEMPERATUR', 'DAMPFDRUCK', 'NIEDERSCHLAGSHOEHE', 'NIEDERSCHLAGSHOEHE_IND', 'SCHNEEHOEHE']
selected_result = ['LUFTTEMPERATUR', 'NIEDERSCHLAGSHOEHE']
selected_pass = ['MESS_DATUM']
vergangenheit=5
timelag = 3

### Filterung
Jetzt werden die einzelnen Daten gefiltert und gejoint.
Dabei wird für jeden Datensatz zunächst der die Verschiebungen der Zeit vorhergenommen und anschließend ein prefix hinzugefügt.
Der Join ist ein full outer join der alle datensätze miteinander anhand der Spalte `MESS_DATUM` gejoint.
Die Felder die übrich geblieben sind werden mit `-999` aufgefüllt.
Die jeweiligen Spalten zum joinen werden jetzt nicht mehr gebraucht und werden gelöscht.
Anschließend werden alle datensätze gelöscht, in denen Daten fehlen.
Als ausgabe kann man sehen, wo und welche daten gelöscht werden.
Zulätzt werden noch die Featcher und Ziel spalten voneinander getrennt

In [10]:
# Zeitverschiebungen hinzugefügt
straubingDatenFiltered = filter_columns(straubingDaten, selected_columns, variants=vergangenheit, extras=selected_result, selected_pass=selected_pass, timelag=timelag)
arberDatenFiltered = filter_columns(arberDaten, selected_columns, variants=vergangenheit, extras=[], selected_pass=selected_pass, timelag=timelag)
schorndorfDatenFiltered = filter_columns(schorndorfDaten, selected_columns, variants=vergangenheit, extras=[], selected_pass=selected_pass, timelag=timelag)

# Prefixes hinzugefügt
straubingDatenFiltered = straubingDatenFiltered.add_prefix('straubing_')
arberDatenFiltered = arberDatenFiltered.add_prefix('arber_')
schorndorfDatenFiltered = schorndorfDatenFiltered.add_prefix('schorndorf_')

# Ausgabe der Länge der einzelnen Datensätze
print(f'Länge des Straubing Datensatzes: {len(straubingDatenFiltered)}')
print(f'Länge des Arber Datensatzes: {len(arberDatenFiltered)}')
print(f'Länge des Schorndorf Datensatzes: {len(schorndorfDatenFiltered)}')
print('#################')

# Merging der Einzelnen Datensätze zu einem neuen (allDatenFiltered)
allDatenFiltered = pd.merge(straubingDatenFiltered, arberDatenFiltered, how='outer', left_on='straubing_MESS_DATUM', right_on='arber_MESS_DATUM').merge(schorndorfDatenFiltered, how='outer', left_on='straubing_MESS_DATUM', right_on='schorndorf_MESS_DATUM')
allDatenFiltered = allDatenFiltered.fillna(-999)
allDatenFiltered = allDatenFiltered.drop(columns=['straubing_MESS_DATUM', 'arber_MESS_DATUM', 'schorndorf_MESS_DATUM'])


# Iteriere über Zeilen und Spalten
for row in range(allDatenFiltered.shape[0]):
    for col in range(allDatenFiltered.shape[1]):
        allDatenFiltered.iloc[row, col] = replace_missing_values(allDatenFiltered.iloc[row, col], allDatenFiltered, row, col)


# Ausgabe der zu löschenden Daten und diese dann auch gelöscht
# Hier wird noch ein unterschied für die prefixes gemacht, dies ist nur noch aus lagacy gründen drinnen
preflen = len(allDatenFiltered)
pd.set_option('display.max_rows', None)
print(f'Anzahl wie viele daten in einer Spalte Fehlen:\n{(allDatenFiltered == -999).sum()}')
pd.reset_option('display.max_rows')
for prefix in ['straubing_', 'arber_', 'schorndorf_']:
    mask = allDatenFiltered.columns.str.startswith(prefix)
    allDatenFiltered = allDatenFiltered[~allDatenFiltered.loc[:, mask].isin([-999]).any(axis=1)]
allDatenFiltered = allDatenFiltered.sample(frac=1).reset_index(drop=True)
print('#################')
print(f'Gesamte Datengröße: {preflen}\nVerlorene Daten durch die Filterung: {preflen - len(allDatenFiltered)}\nFinale Datengröße: {len(allDatenFiltered)}')

# Seperation von Featchers und Zieldaten (x, y)
Y_Daten_Temp = allDatenFiltered.loc[:, [f'straubing_LUFTTEMPERATUR_{vergangenheit+1}']].copy()
X_Daten_Temp = allDatenFiltered.drop(columns=[f'straubing_LUFTTEMPERATUR_{vergangenheit+1}']).drop(columns=[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit+1}'])

Y_Daten_Reg = allDatenFiltered.loc[:, [f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit+1}']].copy()
X_Daten_Reg = allDatenFiltered.drop(columns=[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit+1}']).drop(columns=[f'straubing_LUFTTEMPERATUR_{vergangenheit+1}'])

X_Temp = X_Daten_Temp.values
Y_Temp = Y_Daten_Temp.values.flatten()

X_Reg = X_Daten_Reg.values
Y_Reg = Y_Daten_Reg.values.flatten()

Länge des Straubing Datensatzes: 23741
Länge des Arber Datensatzes: 12114
Länge des Schorndorf Datensatzes: 7305
#################
Anzahl wie viele daten in einer Spalte Fehlen:
straubing_LUFTTEMPERATUR_0             0
straubing_LUFTTEMPERATUR_1             0
straubing_LUFTTEMPERATUR_2             0
straubing_LUFTTEMPERATUR_3             0
straubing_LUFTTEMPERATUR_4             0
straubing_LUFTTEMPERATUR_5             0
straubing_DAMPFDRUCK_0                 0
straubing_DAMPFDRUCK_1                 0
straubing_DAMPFDRUCK_2                 0
straubing_DAMPFDRUCK_3                 0
straubing_DAMPFDRUCK_4                 0
straubing_DAMPFDRUCK_5                 0
straubing_NIEDERSCHLAGSHOEHE_0         0
straubing_NIEDERSCHLAGSHOEHE_1         0
straubing_NIEDERSCHLAGSHOEHE_2         0
straubing_NIEDERSCHLAGSHOEHE_3         0
straubing_NIEDERSCHLAGSHOEHE_4         0
straubing_NIEDERSCHLAGSHOEHE_5         0
straubing_NIEDERSCHLAGSHOEHE_IND_0     0
straubing_NIEDERSCHLAGSHOEHE_IND_1     0
st

## Parameter für den Random Forest

In [11]:
# Importieren der benötigten Bibliotheken
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
def train(X, Y, n_estimators, max_depth, test_size):
    # Train Test split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

    # Initialisieren des Random Forest Regressors
    rf_regressor = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1, max_depth=max_depth)
    
    # Trainieren des Regressors mit den Trainingsdaten
    rf_regressor.fit(X_train, y_train)
    
    # Vorhersagen auf den Testdaten
    y_pred_test = rf_regressor.predict(X_test)
    y_pred_train = rf_regressor.predict(X_train)

    return (mean_squared_error(y_test, y_pred_test), mean_squared_error(y_train, y_pred_train))

In [ ]:
estimators_range = range(5, 100, 5)
depth_range = range(5, 100, 5)
split_range = range(5, 90, 5)

estimators_mse_test_Temp = []
estimators_mse_train_Temp = []
depth_mse_test_Temp = []
depth_mse_train_Temp = []
split_mse_test_Temp = []
split_mse_train_Temp = []
estimators_mse_test_Reg = []
estimators_mse_train_Reg = []
depth_mse_test_Reg = []
depth_mse_train_Reg = []
split_mse_test_Reg = []
split_mse_train_Reg = []

for est in estimators_range:
    mse_test, mse_train = train(X_Temp, Y_Temp, est, 20, 0.2)
    estimators_mse_test_Temp.append(mse_test)
    estimators_mse_train_Temp.append(mse_train)
    
for depth in depth_range:
    mse_test, mse_train = train(X_Temp, Y_Temp, 50, depth, 0.2)
    depth_mse_test_Temp.append(mse_test)
    depth_mse_train_Temp.append(mse_train)
    
for split in split_range:
    mse_test, mse_train = train(X_Temp, Y_Temp, 50, 20, split/100)
    split_mse_test_Temp.append(mse_test)
    split_mse_train_Temp.append(mse_train)

for est in estimators_range:
    mse_test, mse_train = train(X_Reg, Y_Reg, est, 40, 0.2)
    estimators_mse_test_Reg.append(mse_test)
    estimators_mse_train_Reg.append(mse_train)
    
for depth in depth_range:
    mse_test, mse_train = train(X_Reg, Y_Reg, 50, depth, 0.2)
    depth_mse_test_Reg.append(mse_test)
    depth_mse_train_Reg.append(mse_train)
    
for split in split_range:
    mse_test, mse_train = train(X_Reg, Y_Reg, 50, 40, split/100)
    split_mse_test_Reg.append(mse_test)
    split_mse_train_Reg.append(mse_train)

In [ ]:
# Erstellen von Subplots (2 Zeilen, 3 Spalten)
fig, axs = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: Forest Trees
axs[0, 0].plot(estimators_range, estimators_mse_test_Temp, marker='o', label='Test MSE')
axs[0, 0].plot(estimators_range, estimators_mse_train_Temp, marker='o', label='Train MSE')
axs[0, 0].set_ylabel('MSE (Mean Squared Error)')
axs[0, 0].set_xlabel('Trees')
axs[0, 0].set_title('Forest Trees')
axs[0, 0].legend()

# Plot 2: Tree Depth
axs[0, 1].plot(depth_range, depth_mse_test_Temp, marker='o', label='Test MSE')
axs[0, 1].plot(depth_range, depth_mse_train_Temp, marker='o', label='Train MSE')
axs[0, 1].set_ylabel('MSE (Mean Squared Error)')
axs[0, 1].set_xlabel('Depth')
axs[0, 1].set_title('Tree Depth')
axs[0, 1].legend()

# Plot 3: Train Test Split
axs[0, 2].plot([split/100 for split in split_range], split_mse_test_Temp, marker='o', label='Test MSE')
axs[0, 2].plot([split/100 for split in split_range], split_mse_train_Temp, marker='o', label='Train MSE')
axs[0, 2].set_ylabel('MSE (Mean Squared Error)')
axs[0, 2].set_xlabel('Split')
axs[0, 2].set_title('Train Test Split')
axs[0, 2].legend()

# Plot 4: Forest Trees
axs[1, 0].plot(estimators_range, estimators_mse_test_Reg, marker='o', label='Test MSE')
axs[1, 0].plot(estimators_range, estimators_mse_train_Reg, marker='o', label='Train MSE')
axs[1, 0].set_ylabel('MSE (Mean Squared Error)')
axs[1, 0].set_xlabel('Trees')
axs[1, 0].set_title('Forest Trees')
axs[1, 0].legend()

# Plot 5: Tree Depth
axs[1, 1].plot(depth_range, depth_mse_test_Reg, marker='o', label='Test MSE')
axs[1, 1].plot(depth_range, depth_mse_train_Reg, marker='o', label='Train MSE')
axs[1, 1].set_ylabel('MSE (Mean Squared Error)')
axs[1, 1].set_xlabel('Depth')
axs[1, 1].set_title('Tree Depth')
axs[1, 1].legend()

# Plot 6: Train Test Split
axs[1, 2].plot([split/100 for split in split_range], split_mse_test_Reg, marker='o', label='Test MSE')
axs[1, 2].plot([split/100 for split in split_range], split_mse_train_Reg, marker='o', label='Train MSE')
axs[1, 2].set_ylabel('MSE (Mean Squared Error)')
axs[1, 2].set_xlabel('Split')
axs[1, 2].set_title('Train Test Split')
axs[1, 2].legend()

# Layout anpassen
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Train Test split
X_train_Temp, X_test_Temp, y_train_Temp, y_test_Temp = train_test_split(X_Temp, Y_Temp, test_size=0.2)
X_train_Reg, X_test_Reg, y_train_Reg, y_test_Reg = train_test_split(X_Reg, Y_Reg, test_size=0.2)

# Gradient Boosting für die Temperatur-Vorhersagen
gb_regressor_Temp = GradientBoostingRegressor(n_estimators=100,  # Anzahl der Bäume im Ensemble
                                              learning_rate=0.1,  # Lernrate
                                              max_depth=4,  # Maximale Tiefe der Bäume
                                              random_state=42)

# Trainieren des Gradient Boosting-Modells mit den Trainingsdaten
gb_regressor_Temp.fit(X_train_Temp, y_train_Temp)

# Vorhersagen auf den Testdaten
y_pred_test_Temp = gb_regressor_Temp.predict(X_test_Temp)
y_pred_train_Temp = gb_regressor_Temp.predict(X_train_Temp)

# Auswertung des Gradient Boosting-Modells für die Temperatur
print("RMSE Temperatur Vortag: " + str(mean_squared_error(Y_Daten_Temp, X_Daten_Temp[f'straubing_LUFTTEMPERATUR_{vergangenheit}'])))
print("RMSE Temperatur Test:   " + str(mean_squared_error(y_test_Temp, y_pred_test_Temp)))
print("RMSE Temperatur Train:  " + str(mean_squared_error(y_train_Temp, y_pred_train_Temp)))

# Gradient Boosting für die Niederschlags-Vorhersagen
gb_regressor_Reg = GradientBoostingRegressor(n_estimators=100,  # Anzahl der Bäume im Ensemble
                                             learning_rate=0.1,  # Lernrate
                                             max_depth=4,  # Maximale Tiefe der Bäume
                                             random_state=42)

# Trainieren des Gradient Boosting-Modells mit den Trainingsdaten
gb_regressor_Reg.fit(X_train_Reg, y_train_Reg)

# Vorhersagen auf den Testdaten
y_pred_test_Reg = gb_regressor_Reg.predict(X_test_Reg)
y_pred_train_Reg = gb_regressor_Reg.predict(X_train_Reg)

# Auswertung des Gradient Boosting-Modells für den Niederschlag
print("RMSE Niederschlag Vortag: " + str(mean_squared_error(Y_Daten_Reg, X_Daten_Reg[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit}'])))
print("RMSE Niederschlag Test:   " + str(mean_squared_error(y_test_Reg, y_pred_test_Reg)))
print("RMSE Niederschlag Train:  " + str(mean_squared_error(y_train_Reg, y_pred_train_Reg)))


In [ ]:
from sklearn.ensemble import BaggingRegressor

# Train Test split
X_train_Temp, X_test_Temp, y_train_Temp, y_test_Temp = train_test_split(X_Temp, Y_Temp, test_size=0.2)
X_train_Reg, X_test_Reg, y_train_Reg, y_test_Reg = train_test_split(X_Reg, Y_Reg, test_size=0.2)

# Bagging für die Temperatur-Vorhersagen
bagging_regressor_Temp = BaggingRegressor(estimator=RandomForestRegressor(n_estimators=50, n_jobs=-1, max_depth=20),
                                          n_estimators=10,  # Anzahl der Modelle im Ensemble
                                          random_state=42)

# Trainieren des Bagging-Modells mit den Trainingsdaten
bagging_regressor_Temp.fit(X_train_Temp, y_train_Temp)

# Vorhersagen auf den Testdaten
y_pred_test_Temp = bagging_regressor_Temp.predict(X_test_Temp)
y_pred_train_Temp = bagging_regressor_Temp.predict(X_train_Temp)

# Auswertung des Bagging-Modells für die Temperatur
print("RMSE Temperatur Vortag: " + str(mean_squared_error(Y_Daten_Temp, X_Daten_Temp[f'straubing_LUFTTEMPERATUR_{vergangenheit}'])))
print("RMSE Temperatur Test:   " + str(mean_squared_error(y_test_Temp, y_pred_test_Temp)))
print("RMSE Temperatur Train:  " + str(mean_squared_error(y_train_Temp, y_pred_train_Temp)))

# Bagging für die Niederschlags-Vorhersagen
bagging_regressor_Reg = BaggingRegressor(estimator=RandomForestRegressor(n_estimators=50, n_jobs=-1, max_depth=40),
                                         n_estimators=10,  # Anzahl der Modelle im Ensemble
                                         random_state=42)

# Trainieren des Bagging-Modells mit den Trainingsdaten
bagging_regressor_Reg.fit(X_train_Reg, y_train_Reg)

# Vorhersagen auf den Testdaten
y_pred_test_Reg = bagging_regressor_Reg.predict(X_test_Reg)
y_pred_train_Reg = bagging_regressor_Reg.predict(X_train_Reg)

# Auswertung des Bagging-Modells für den Niederschlag
print("RMSE Niederschlag Vortag: " + str(mean_squared_error(Y_Daten_Reg, X_Daten_Reg[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit}'])))
print("RMSE Niederschlag Test:   " + str(mean_squared_error(y_test_Reg, y_pred_test_Reg)))
print("RMSE Niederschlag Train:  " + str(mean_squared_error(y_train_Reg, y_pred_train_Reg)))


In [ ]:

# Train Test split
X_train_Temp, X_test_Temp, y_train_Temp, y_test_Temp = train_test_split(X_Temp, Y_Temp, test_size=0.2)
X_train_Reg, X_test_Reg, y_train_Reg, y_test_Reg = train_test_split(X_Reg, Y_Reg, test_size=0.2)

# Initialisieren des Random Forest Regressors
rf_regressor_Temp = RandomForestRegressor(n_estimators=50, n_jobs=-1, max_depth=20)
rf_regressor_Reg = RandomForestRegressor(n_estimators=50, n_jobs=-1, max_depth=40)
    
# Trainieren des Regressors mit den Trainingsdaten
rf_regressor_Temp.fit(X_train_Temp, y_train_Temp)
rf_regressor_Reg.fit(X_train_Reg, y_train_Reg)
    
# Vorhersagen auf den Testdaten
y_pred_test_Temp = rf_regressor_Temp.predict(X_test_Temp)
y_pred_train_Temp = rf_regressor_Temp.predict(X_train_Temp)
y_pred_test_Reg = rf_regressor_Reg.predict(X_test_Reg)
y_pred_train_Reg = rf_regressor_Reg.predict(X_train_Reg)
    
# Auswertung des Modells (z.B. Mean Squared Error für Regression)
print("RMSE Temperatur Vortag:" + str(mean_squared_error(Y_Daten_Temp, X_Daten_Temp[f'straubing_LUFTTEMPERATUR_{vergangenheit}'])))
print("RMSE Temperatur Test:  " + str(mean_squared_error(y_test_Temp, y_pred_test_Temp)))
print("RMSE Temperatur Train: " + str(mean_squared_error(y_train_Temp, y_pred_train_Temp)))
print("RMSE Niderschlag Vortag:" + str(mean_squared_error(Y_Daten_Reg, X_Daten_Reg[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit}'])))
print("RMSE Niderschlag Test:  " + str(mean_squared_error(y_test_Reg, y_pred_test_Reg)))
print("RMSE Niderschlag Train: " + str(mean_squared_error(y_train_Reg, y_pred_train_Reg)))

In [ ]:
# Scatterplots nebeneinander darstellen
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# Scatterplot im Traindatensatz
axs[0, 0].scatter(Y_Daten_Temp, X_Daten_Temp[f'straubing_LUFTTEMPERATUR_{vergangenheit}'], alpha=0.5, label='Verhältnist zur vortags Temperatur')
axs[0, 0].scatter(y_train_Temp, y_pred_train_Temp, alpha=0.5, label='Vorhersagen')
axs[0, 0].plot(axs[0, 0].get_xlim(), axs[0, 0].get_xlim(), color='red', linestyle='--', label='Ideallinie')
axs[0, 0].set_xlabel('Tatsächliche Werte')
axs[0, 0].set_ylabel('Vorhersagen')
axs[0, 0].set_title('Vergleich der tatsächlichen Werte und Vorhersagen im Traindatensatz')
axs[0, 0].legend()

# Scatterplot im Testdatensatz
axs[0, 1].scatter(Y_Daten_Temp, X_Daten_Temp[f'straubing_LUFTTEMPERATUR_{vergangenheit}'], alpha=0.5, label='Verhältnist zur vortags Temperatur')
axs[0, 1].scatter(y_test_Temp, y_pred_test_Temp, alpha=0.5, label='Vorhersagen')
axs[0, 1].plot(axs[0, 1].get_xlim(), axs[0, 1].get_xlim(), color='red', linestyle='--', label='Ideallinie')
axs[0, 1].set_xlabel('Tatsächliche Werte')
axs[0, 1].set_ylabel('Vorhersagen')
axs[0, 1].set_title('Vergleich der tatsächlichen Werte und Vorhersagen im Testdatensatz')
axs[0, 1].legend()

# Scatterplot im Traindatensatz
axs[1, 0].scatter(Y_Daten_Reg, X_Daten_Reg[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit}'], alpha=0.5, label='Verhältnist zur vortags Temperatur')
axs[1, 0].scatter(y_train_Reg, y_pred_train_Reg, alpha=0.5, label='Vorhersagen')
axs[1, 0].plot(axs[1, 0].get_xlim(), axs[1, 0].get_xlim(), color='red', linestyle='--', label='Ideallinie')
axs[1, 0].set_xlabel('Tatsächliche Werte')
axs[1, 0].set_ylabel('Vorhersagen')
axs[1, 0].set_title('Vergleich der tatsächlichen Werte und Vorhersagen im Traindatensatz')
axs[1, 0].legend()

# Scatterplot im Testdatensatz
axs[1, 1].scatter(Y_Daten_Reg, X_Daten_Reg[f'straubing_NIEDERSCHLAGSHOEHE_{vergangenheit}'], alpha=0.5, label='Verhältnist zur vortags Temperatur')
axs[1, 1].scatter(y_test_Reg, y_pred_test_Reg, alpha=0.5, label='Vorhersagen')
axs[1, 1].plot(axs[1, 1].get_xlim(), axs[1, 1].get_xlim(), color='red', linestyle='--', label='Ideallinie')
axs[1, 1].set_xlabel('Tatsächliche Werte')
axs[1, 1].set_ylabel('Vorhersagen')
axs[1, 1].set_title('Vergleich der tatsächlichen Werte und Vorhersagen im Testdatensatz')
axs[1, 1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Unterschied der Arrays berechnen
runden = 1
diff_array_test_Temp = np.abs(np.round(y_test_Temp * runden) / runden - np.round(y_pred_test_Temp * runden) / runden)
diff_array_train_Temp = np.abs(np.round(y_train_Temp * runden) / runden - np.round(y_pred_train_Temp * runden) / runden)
diff_array_test_Reg = np.abs(np.round(y_test_Reg * runden) / runden - np.round(y_pred_test_Reg * runden) / runden)
diff_array_train_Reg = np.abs(np.round(y_train_Reg * runden) / runden - np.round(y_pred_train_Reg * runden) / runden)

unique_elements_test_Temp, counts_test_Temp = np.unique(diff_array_test_Temp, return_counts=True)
unique_elements_train_Temp, counts_train_Temp = np.unique(diff_array_train_Temp, return_counts=True)
unique_elements_test_Reg, counts_test_Reg = np.unique(diff_array_test_Reg, return_counts=True)
unique_elements_train_Reg, counts_train_Reg = np.unique(diff_array_train_Reg, return_counts=True)

# Werte-Bereich mit dem gewünschten Abstand erstellen
max_value_Temp = max(max(unique_elements_test_Temp), max(unique_elements_train_Temp))
max_value_Reg = max(max(unique_elements_test_Reg), max(unique_elements_train_Reg))
def expand(min, max, step, val, count):
    all_values = np.arange(min, max + step, step)
    # Index-Array für vorhandene Werte erstellen
    existing_values_idx = np.isin(all_values, val)
    # Fehlende Werte mit Häufigkeit 0 einfügen
    all_frequencies = np.zeros_like(all_values, dtype=int)
    all_frequencies[existing_values_idx] = count
    return (all_values, all_frequencies)

unique_elements_test_Temp, counts_test_Temp = expand(0, max_value_Temp, 1/runden, unique_elements_test_Temp, counts_test_Temp)
unique_elements_train_Temp, counts_train_Temp = expand(0, max_value_Temp, 1/runden, unique_elements_train_Temp, counts_train_Temp)
unique_elements_test_Reg, counts_test_Reg = expand(0, max_value_Reg, 1/runden, unique_elements_test_Reg, counts_test_Reg)
unique_elements_train_Reg, counts_train_Reg = expand(0, max_value_Reg, 1/runden, unique_elements_train_Reg, counts_train_Reg)


# Verteilung plotten als Bar Plot
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(20, 14))

# Breite der Bars
bar_width = 0.5

# Positionen der x-Ticks für die beiden Gruppen
x_ticks_test_Temp = np.arange(len(unique_elements_test_Temp))
x_ticks_train_Temp = np.arange(len(unique_elements_train_Temp))
x_ticks_test_Reg = np.arange(len(unique_elements_test_Reg))
x_ticks_train_Reg = np.arange(len(unique_elements_train_Reg))

# Plot für Testdaten
bar_test_Temp = ax1.bar(x_ticks_test_Temp, counts_test_Temp, width=bar_width, label="Test Daten")
ax1.set_xlabel('Grad')
ax1.set_ylabel('Häufigkeit')
ax1.set_xticks(x_ticks_test_Temp)
ax1.set_xticklabels(unique_elements_test_Temp)
ax1.legend()
ax1.set_title('Verteilung des absoluten Fehlers der Testdaten')

# Text über den Bars anzeigen
for bar, counts in zip(bar_test_Temp, counts_test_Temp):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width() / 2, height, f'{counts}', 
            ha='center', va='bottom')

# Plot für Trainingsdaten
bar_train_Temp = ax2.bar(x_ticks_train_Temp, counts_train_Temp, width=bar_width, label="Trainings Daten")
ax2.set_xlabel('Grad')
ax2.set_ylabel('Häufigkeit')
ax2.set_xticks(x_ticks_train_Temp)
ax2.set_xticklabels(unique_elements_train_Temp)
ax2.legend()
ax2.set_title('Verteilung des absoluten Fehlers der Trainingsdaten')

# Text über den Bars anzeigen
for bar, counts in zip(bar_train_Temp, counts_train_Temp):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width() / 2, height, f'{counts}', 
            ha='center', va='bottom')


# Plot für Testdaten
bar_test_Reg = ax3.bar(x_ticks_test_Reg, counts_test_Reg, width=bar_width, label="Test Daten")
ax3.set_xlabel('Grad')
ax3.set_ylabel('Häufigkeit')
ax3.set_xticks(x_ticks_test_Reg)
ax3.set_xticklabels(unique_elements_test_Reg)
ax3.legend()
ax3.set_title('Verteilung des absoluten Fehlers der Testdaten')

# Text über den Bars anzeigen
for bar, counts in zip(bar_test_Reg, counts_test_Reg):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width() / 2, height, f'{counts}', 
            ha='center', va='bottom')

# Plot für Trainingsdaten
bar_train_Reg = ax4.bar(x_ticks_train_Reg, counts_train_Reg, width=bar_width, label="Trainings Daten")
ax4.set_xlabel('Grad')
ax4.set_ylabel('Häufigkeit')
ax4.set_xticks(x_ticks_train_Reg)
ax4.set_xticklabels(unique_elements_train_Reg)
ax4.legend()
ax4.set_title('Verteilung des absoluten Fehlers der Trainingsdaten')

# Text über den Bars anzeigen
for bar, counts in zip(bar_train_Reg, counts_train_Reg):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width() / 2, height, f'{counts}', 
            ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Extrahieren der Importance der Trees
importances_Temp = np.round(rf_regressor_Temp.feature_importances_*100, 2)
std_Temp = np.round(np.std([tree.feature_importances_ for tree in rf_regressor_Temp.estimators_], axis=0)*100, 2)
importances_Reg = np.round(rf_regressor_Reg.feature_importances_*100, 2)
std_Reg = np.round(np.std([tree.feature_importances_ for tree in rf_regressor_Reg.estimators_], axis=0)*100, 2)

# Indizes der sortierten Importance-Werte
sorted_indices_Temp = np.argsort(importances_Temp)
sorted_indices_Reg = np.argsort(importances_Reg)

# Sortieren der Importance und Standardabweichung
importances_Temp = importances_Temp[sorted_indices_Temp]
std_Temp = std_Temp[sorted_indices_Temp]
importances_Reg = importances_Reg[sorted_indices_Reg]
std_Reg = std_Reg[sorted_indices_Reg]

forest_importances_Temp = pd.Series(importances_Temp, index=X_Daten_Temp.columns[sorted_indices_Temp])
forest_importances_Reg = pd.Series(importances_Reg, index=X_Daten_Reg.columns[sorted_indices_Reg])

# Breite des Bildes anpassen
fig, ax = plt.subplots(figsize=(8, 30))

# Balkenplot mit Fehlerbalken
forest_importances_Temp.plot.barh(xerr=std_Temp, ax=ax, capsize=4)

# Titel und Achsenbeschriftungen
ax.set_title("Feature importances with uncertainty")
ax.set_xlabel("Mean decrease in impurity")

# Logarithmische Skala für die x-Achse
ax.set_xscale('log')

# Werte neben den Balken anzeigen
for i, (feature, importance, uncertainty) in enumerate(zip(forest_importances_Temp.index, forest_importances_Temp.values, std_Temp)):
    ax.text(importance + uncertainty + 0.1, i, f'{importance:.2f}% ±{uncertainty:.2f}%', ha='left', va='center')

plt.show()

# Breite des Bildes anpassen
fig, ax = plt.subplots(figsize=(8, 30))

# Balkenplot mit Fehlerbalken
forest_importances_Reg.plot.barh(xerr=std_Reg, ax=ax, capsize=4)

# Titel und Achsenbeschriftungen
ax.set_title("Feature importances with uncertainty")
ax.set_xlabel("Mean decrease in impurity")

# Logarithmische Skala für die x-Achse
ax.set_xscale('log')

# Werte neben den Balken anzeigen
for i, (feature, importance, uncertainty) in enumerate(zip(forest_importances_Reg.index, forest_importances_Reg.values, std_Reg)):
    ax.text(importance + uncertainty + 0.1, i, f'{importance:.2f}% ±{uncertainty:.2f}%', ha='left', va='center')

plt.show()